### Kernel smoothing (with Python)

In [ ]:
import numpy as np
import scipy
from matplotlib import pyplot as plt

#### True function

Our "true function" that we will try to recover from noisy data:

In [ ]:
def true_function(x):
    return .3 * np.sin(4*x) / (1+(2*x)**2)  + 10*x**2 / (1+(4*x)**4) 

In [ ]:
xg = np.linspace( -2, 2, num=1000 )

In [ ]:
plt.scatter( xg, true_function(xg), s=.1 )

#### Noisy data

Draw $n$ random x values from the uniform distribution over $[-2;2]$

In [ ]:
n = 300
x = np.random.uniform( -2, 2, n)

Calculate true function values at these positions and distort them by adding noise drawn from a normal distribution with mean 0 and some small standard deviation

In [ ]:
y = true_function(x) + np.random.normal( 0, .02, n )

Plot the simulated data

In [ ]:
plt.scatter( x, y, s=.5 )

#### Smoothing kernel

We use the tricube kernel:

In [ ]:
def tricube(x):
    return np.where( np.abs(x)<1, (1-np.abs(x)**3)**3, 0 )

Plot it, to check its shape

In [ ]:
plt.scatter( xg, tricube(xg), s=.1 )

#### Simple kernel smoothing

##### Imperative style

First, as `for` loop (imperative programming style)

In [ ]:
# Our smoothing bandwidth (kernel width):
h = .2

# We want to get smoothed values at the following x positions:
xsm = np.linspace( -2, 2, num=300 )

# The values will go here:
ysm = np.empty( len(xsm) )

# Go through all these:
for i in range( len(xsm) ):
    
    # Calculate smoothed y value for x position xsm[i]
    # For this, go through all data points
    weighted_sum = 0.
    weight_sum = 0.
    for j in range( len(x) ):
        
        # Calculate distance of query position to data position
        d = x[j] - xsm[i]
        # Get weight of data point
        w = tricube(d / h)

        # Add to sums
        weighted_sum += w * y[j]
        weight_sum += w

    # Store result
    ysm[i] = weighted_sum / weight_sum


# Plot result (in red)
plt.scatter( xsm, ysm, s=.2, c="red" )
# Add original "true function" in blue
plt.scatter( xg, true_function(xg), s=.2, c="blue" )
# Add noisy data in black
plt.scatter( x, y, s=.4, c="black" )

##### Semi-functional style

Now, we write this in a "functional-programming" style.

We first write a function that takes a query position `x0` (taking the rolw of `xsm[i]` in the code above) and calculates the weighted average of the data values `y` with weights for a kernel centered around `x0` within the data positions `x`:

In [ ]:
def simple_smooth( x0, x, y, h ):
    # Here, x0 and h are scalars; and x and y are vectors

    weighted_sum = 0.
    weight_sum = 0.
    for j in range( len(x) ):
        
        d = x[j] - x0
        w = tricube(d / h)

        weighted_sum += w * y[j]
        weight_sum += w

    
    return weighted_sum / weight_sum

We want to get smoothed values at the following positions:

In [ ]:
xsm = np.linspace( -2, 2, num=300 )

We call `simple_smooth` for each of these positions

In [ ]:
ysm = np.empty( len(xsm) )
for i in range(len(xsm)):
    ysm[i] = simple_smooth( xsm[i], x, y, h )

We get the same result as before

In [ ]:
plt.scatter( xsm, ysm, s=.2, c="red" )

##### Functional style

It would be nice if we could simply pass the whole vector `xsm` to `simple_smooth` rather than each value separately.

This can be achieved by "vectorizing" the function

In [ ]:
def simple_smooth_vectorized( x0, x, y, h ):
    return np.vectorize( lambda x0: simple_smooth( x0, x, y, h ) )( xsm )

This is, in essence, just a fancy way of writing this:

In [ ]:
def simple_smooth_vectorized( x0, x, y, h ):
    ans = np.empty_like(x0)
    for i in range( len(x0) ):
        ans[i] = simple_smooth( x0[i], x, y, h )
    return ans

In [ ]:
xsm = np.linspace( -2, 2, num=300 )
ysm = simple_smooth_vectorized( xsm, x, y, h )
plt.scatter( xsm, ysm, s=.2, c="red" )

#### Regression smoothing

To develop the code, we first chose a fixed query position `x0`

In [ ]:
x0 = .35

We calculate the weights around the query position for all data positions

In [ ]:
w = tricube( (x-x0)/h )

Here is the weighted data:

In [ ]:
plt.scatter( x, y, s=3*w )

Now let's fit a parabola to this.

We first make a design matrix $M$ (usually $X$ in regression setting, here called `mm`) by stacking the following three columns:
- $\mathbf{1}$ (a column of 1s),
- $\mathbf{x}$ (the column vector of data positions), and
- $\mathbf{x}^{\odot 2}$ (the column vector of squared data positions)

In [ ]:
mm = np.column_stack((np.ones_like(x), x, x**2))

mm[:5,]  # Show first 5 rows

Now, solve the normal equations for weighted linear regression, $M^T W M \boldsymbol{\upbeta} = M^T W \mathbf{y}$, where $W=\operatorname{diag}W$

In [ ]:
beta = scipy.linalg.solve( mm.T @ np.diag(w) @ mm, mm.T @ np.diag(w) @ y )
beta

Now, construct a matrix like `mm` before but not using the data positions `x` but the linear sequence of query positions, `xsm`. Multiply this with $\boldsymbol{\upbeta}$ to get the smoothed values

In [ ]:
ysm = np.column_stack((np.ones_like(xsm), xsm, xsm**2)) @ beta

Plot the parabola (in magenta), with the data points in blue on top,

In [ ]:
plt.scatter( xsm, ysm, s=.1, c="magenta" )

plt.scatter( x, y, s=3*w )
plt.vlines( x0, .4, .5, color="gray" )
plt.ylim( 0, .6 )

Calculate the y value of the parabola at the query position

In [ ]:
y0 = np.dot( beta, np.array([ 1, x0, x0**2 ]) )
y0

Put all this together into one function:

In [ ]:
def qureg_smooth( x0, x, y, h ):
    w = tricube( (x-x0)/h )
    mm = np.column_stack((np.ones_like(x), x, x**2))
    beta = scipy.linalg.solve( mm.T @ np.diag(w) @ mm, mm.T @ np.diag(w) @ y )
    return np.dot( beta, np.array([ 1, x0, x0**2 ]) )

The following version takes a vector of query positions rather than a single position:

In [ ]:
def qureg_smooth_vectorized( xq, x, y, h ):
    ans = np.empty_like(xq)
    mm = np.column_stack((np.ones_like(x), x, x**2))
    for i in range(len(xq)):
        w = tricube( (x-xq[i])/h )        
        beta = scipy.linalg.solve( mm.T @ np.diag(w) @ mm, mm.T @ np.diag(w) @ y )
        ans[i] = np.dot( beta, np.array([ 1, xq[i], xq[i]**2 ]) )
    return ans

In [ ]:
# Plot true function and data as background
plt.scatter( xg, true_function(xg), s=.2, c="blue" )
plt.scatter( x, y, s=.4, c="black" )

# Perform smoothing and plot result
xsm = np.linspace( -2, 2, num=300 )
ysm = qureg_smooth_vectorized( xsm, x, y, h )
plt.scatter( xsm, ysm, s=.2, c="red" )